Our goal is to use this industrial scale dataset to predict if a customer will default in the future. We aim to put together a quick and rough solution.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn import metrics
import gc

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

This data set is too large to be loaded as a csv file. We convert the data from csv to parquet and then load the parquet format into a pandas dataframe. 

Next, we take a preview of our data.

In [1]:
import pandas as pd
df=pd.read_parquet('/kaggle/input/amex-parquet/train_data.parquet')
print(df.head)

<bound method NDFrame.head of                                                customer_ID         S_2  \
0        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2017-03-09   
1        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2017-04-07   
2        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2017-05-28   
3        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2017-06-13   
4        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2017-07-16   
...                                                    ...         ...   
5531446  fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...  2017-11-05   
5531447  fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...  2017-12-23   
5531448  fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...  2018-01-06   
5531449  fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...  2018-02-06   
5531450  fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...  2018-03-14   

              P_2      D_39       B_1       B_2       R_1       S_3      D_41  \


Due to hardware limitations we will need to reduce the size of our data. We note the customer ID and the date columns. 

Lets separate our labels from our dataset.

In [3]:
print(len(df.columns))
df_label = df.iloc[:,len(df.columns)-1]
df = df.iloc[:,0:len(df.columns)-1]
print(df_label.sum()/len(df_label))

190
0.06119057300697412


Our train data contains % positive cases.

Now lets take a look at our data types.

In [3]:
print(df.dtypes.value_counts())

float32    185
object       4
int64        1
dtype: int64


There are four non-numeric data types. Let's convert these so we may apply a logistic regression model.

Let's plot our columns.

In [4]:
#hist = df1.hist(bins=10, figsize = (40,200), layout=(-1,4) )

Let's find the columns with a significant amount of NaN values. 

In [ ]:
temp = [column for column in df.columns if df[column].isnull().sum()/len(df) >= 0.9]
print(len(temp))

#drop columns with high freq of NaN
df.drop(temp, axis=1, inplace=True)

print( len(df.columns) )

In [ ]:
#using only most recent transaction from each customer
temp = df.shape
df=df.set_index(['customer_ID'])
df=df.ffill()
df=df.bfill()
df=df.reset_index()

df=df.groupby('customer_ID').tail(1)
df=df.set_index(['customer_ID'])

#Drop date column since it is no longer useful
df.drop(['S_2'],axis=1,inplace=True)

print(temp, df.shape)

#inspecting NaN
print('Columns left with NaN:')
temp = [column for column in df.columns if df[column].isnull().sum()/len(df) >0]
print(len(temp))

In [11]:
print(df.head)
keep = df.columns

<bound method NDFrame.head of                                                customer_ID         S_2  \
0        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2017-03-09   
1        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2017-04-07   
2        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2017-05-28   
3        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2017-06-13   
4        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2017-07-16   
...                                                    ...         ...   
5531446  fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...  2017-11-05   
5531447  fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...  2017-12-23   
5531448  fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...  2018-01-06   
5531449  fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...  2018-02-06   
5531450  fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...  2018-03-14   

              P_2      D_39       B_1       B_2       R_1       S_3      D_41  \


We aim to remove data columns that have too high a correlation as logistic regression requires it.

In [ ]:
print(df.shape)
# Create correlation matrix
corr_matrix = df.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find features with correlation greater than 0.9
to_drop = [column for column in upper.columns if any(upper[column] > 0.9)]

# Drop features w/ high correl
df.drop(to_drop, axis=1, inplace=True)

print(df.shape)

(5531451, 189)


To further reduce the degrees of freedom we remove columns with low variance

In [ ]:
#Removing low variance columns in interest of ram
from sklearn.feature_selection import VarianceThreshold
from itertools import compress

temp = df.drop(['D_63', 'D_64'], axis=1)

# Initialize and fit the method
vt = VarianceThreshold(threshold = float(0.1))
vt.fit(temp)

#columns with sufficient variance
keep = list(compress(temp.columns, vt.get_support()))

keep.append('D_63')
keep.append('D_64')

df=df[keep]

keep.append('customer_ID')
keep.append('S_2')
len(keep)

In [ ]:
""""
#removing outliers
print(df1.shape)

df1 = df1[df1['R_6'] < df1['R_6'].quantile(0.97)]
print(df1['R_6'].max())
print(df1.shape)"""

In [ ]:
"""#df1.iloc[:100000,7].value_counts()
print(df1.iloc[:,1].head())


#What type of variable for dates
df1['S_2'] = pd.to_datetime(df1['S_2'])
df1['S_2'] = pd.to_numeric(df1['S_2'])

#normalizing
#df1['S_2'] = (df1['S_2']-df1['S_2'].min())/(df1['S_2'].max() - df1['S_2'].min())
print(df1['S_2'].head())

df1['S_2'] = pd.to_timedelta(df1['S_2'])
print(df1.iloc[:,1].dt.total_seconds())
"""

Finally we convert our object columns to numeric via hot ones

In [ ]:
#Hot ones
df1 = pd.get_dummies(df1)
print(df1.shape)
print(df1.columns)
print(df1['D_64_-1'].sum())
df1.drop(['D_64_-1'], axis=1, inplace = True)
print(df1.shape)

In [ ]:
#Handling missing values
#my_imputer = SimpleImputer()
#df1.iloc[:,:] = my_imputer.fit_transform(df1.iloc[:,:])

In [ ]:
X = df1.iloc[:, :].values.reshape(-1, len(df1.columns))
Y = df_label.iloc[:len(df1), 1].values.reshape(-1, 1)

In [ ]:
"""
# create object for the class
log = LogisticRegression()
log.fit(X, Y) 
Y_pred = log.predict(X)

print(Y_pred, np.sum(Y_pred))
print(log.score(X, Y))"""

In [ ]:

#trying random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

model = RandomForestClassifier(n_estimators=400, max_features='sqrt', bootstrap=True, max_depth=30, min_samples_leaf=1, min_samples_split=5, n_jobs=-1)
#rf_random = GridSearchCV(estimator = rf, param_grid = random_grid, cv = 3, verbose=1, n_jobs = -1)
# Fit the random search model
model.fit(X,Y)

In [ ]:
Y_pred = model.predict_proba(X)
Y_pred = Y_pred[:,1]
print(Y_pred.shape, Y_pred[25:50])

In [ ]:
"""from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV

param_random_gb = {'learning_rate': np.arange(0.05,0.55, 0.1), 'n_estimators' : [125,150,175], 'subsample' : np.arange(0.3,1.0, 0.1), 'max_depth':[3,4,5]}

mse_random = RandomizedSearchCV(estimator = XGBClassifier(), param_distributions = param_random_gb, n_iter = 10,scoring = 'neg_mean_squared_error', cv = 4, verbose = 1)



mse_random.best_params_={'subsample': 0.5, 'n_estimators': 175, 'max_depth': 3, 'learning_rate': 0.15}
mse_random.best_score_ = (0.32263831733224874)**2

mse_random.fit(X,Y)

#Run XGBoost model with the best parameters found
model=XGBClassifier(n_estimators=200,max_depth=3,learning_rate=0.15, subsample=0.5)
model.fit(x_train_split,y_train_split)
#Test the model
y_predict=model.predict(x_test_split)
print('XGBoost Classifier Accuracy: {:.3f}'.format(accuracy_score(y_test_split, y_predict)))"""

In [ ]:
"""def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
accuracy = evaluate(model, X, Y)

print('Accuracy:{:0.2f}%.'.format( 100 * accuracy))"""

In [ ]:
""""cm = metrics.confusion_matrix(Y, Y_pred)

plt.figure(figsize=(9,9))
sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'YlGnBu');
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
all_sample_title = 'Accuracy Score: {0}'.format(round(log.score(X, Y),3))
plt.title(all_sample_title, size = 15);

print('Accuracy:',round(metrics.accuracy_score(Y, Y_pred),3))
metrics.roc_curve(Y, Y_pred)
print(metrics.roc_auc_score(Y, Y_pred))"""

In [ ]:
#free up ram
del df1, df_label
gc.collect()

In [ ]:
#run prediction on test data

print(len(keep))
#need to only load some columns due to ram limitations
df2=pd.read_parquet('/kaggle/input/amex-parquet/test_data.parquet', columns =keep)
print(df2.shape)

In [ ]:
df2.head()

In [ ]:
#Handling missing values via imputation
#df2.iloc[:,:] = my_imputer.fit_transform(df2.iloc[:,:])

In [ ]:
#using only most recent transaction from each customer
#Does this make sense for test data??
temp = df2.shape
df2=df2.set_index(['customer_ID'])
df2=df2.ffill()
df2=df2.bfill()
df2=df2.reset_index()

df2=df2.groupby('customer_ID').tail(1)
df2=df2.set_index(['customer_ID'])

#Drop date column since it is no longer useful
df2.drop(['S_2'],axis=1,inplace=True)

print(temp, df2.shape)

#inspecting NaN
print('Columns left with NaN:')
for i in range(len(df2.columns)):
    if (df2.iloc[:,i].isnull().sum()/len(df2) > 0):
        print(df2.columns[i], round(df2.iloc[:,i].isnull().sum()/len(df2),2))

In [ ]:
#hot ones
df2 = pd.get_dummies(df2)
print(df2.shape)
print(df2.columns)

In [ ]:
df2.head()

In [ ]:
X = df2.iloc[:, :].values.reshape(-1, len(df2.columns))

In [ ]:
"""X = df2.iloc[:, :].values.reshape(-1, len(df2.columns))

Y_pred2 = log.predict(X)
print(Y_pred2)

df2 = df2.reset_index()

final = pd.DataFrame({"customer_ID":df2.customer_ID,"prediction":Y_pred2})

final.to_csv('submission.csv', index=False)
print(Y_pred2, np.sum(Y_pred2))

#score ended as ~50"""

In [ ]:
Y_pred2 = model.predict_proba(df2)
Y_pred2 = Y_pred2[:,1]
df2 = df2.reset_index()
print(Y_pred2, np.sum(Y_pred2))
final = pd.DataFrame({"customer_ID":df2.customer_ID,"prediction":Y_pred2})
print(final)

In [ ]:
final.to_csv('submission.csv', index=False)
print(final)